In [22]:
import os
import random
random.seed(1234)

def get_subfolders_dictionary(root_path):
    images_list = []
    labels_dict = {}
    current_label = 0

    for subfolder in os.listdir(root_path):
        if os.path.isdir(os.path.join(root_path, subfolder)):
            images = [image for image in os.listdir(os.path.join(root_path, subfolder)) if image.endswith(('.jpg'))]
            for img in images:

                # Gerando um número único para cada nome da pasta de imagens
                if subfolder in labels_dict:
                    label = labels_dict[subfolder]
                else:
                    label = current_label
                    labels_dict[subfolder] = label
                    current_label += 1

                # Lista irá conter dicionário com o nome da subpasta, nome da imagem e o label dela (int)
                images_list.append({"subfolder": subfolder, "img": img, "label": label})
    return images_list


def split_dataset(images_list, train_ratio=0.8):

    # Embaralhar os caminhos das imagens
    random.shuffle(images_list)

    # Calcular o tamanho do conjunto de treinamento
    num_train = int(len(images_list) * train_ratio)

    # Dividir os caminhos das imagens em conjuntos de treinamento e teste
    train_set = images_list[:num_train]
    test_set = images_list[num_train:]

    return train_set, test_set

In [23]:
# Imports:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import time
import os
import copy
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [24]:
# Clear cuda cache before starting:
torch.cuda.empty_cache()
cudnn.benchmark = True

class CustomImageDataset(Dataset):
    def __init__(self, images_list, root_path, transform=None):
        self.images_list = images_list
        self.transform = transform
        self.root_path = root_path

    def __len__(self):
        return len(self.images_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_path, self.images_list[idx]['subfolder'], self.images_list[idx]['img'])
        image = Image.open(img_name).convert('RGB')
        label = self.images_list[idx]['label']

        if self.transform:
            image = self.transform(image)

        return image, label

def resnet18_data_preprocessing(images_list, root_path, batch_size=32, workers=2, device='cuda:0'):
    # Define the standard size for ResNet18
    RESNET18_RESIZE = 100
    RESNET_NORMALIZE_VALUES = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    # Compose data transformations:
    data_transforms = transforms.Compose([
        transforms.Resize((RESNET18_RESIZE, RESNET18_RESIZE)),
        transforms.ToTensor(),
        transforms.Normalize(*RESNET_NORMALIZE_VALUES)
    ])

    # Create datasets
    train_dataset = CustomImageDataset(images_list[0], root_path, transform=data_transforms)
    test_dataset = CustomImageDataset(images_list[1], root_path, transform=data_transforms)

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)

    # Dataset sizes and class names
    dataset_sizes = {'train': len(train_dataset), 'test': len(test_dataset)}
    class_names = list(set(img['label'] for img in images_list[0] + images_list[1]))

    # Config device
    device = torch.device(device if torch.cuda.is_available() else "cpu")

    return {'train': train_loader, 'test': test_loader}, dataset_sizes, class_names, device

In [25]:
# Preparação dos dados
root_path = "D:/Unifesp/IA/termografia/termography/dataset"
images_list = get_subfolders_dictionary(root_path)
train_dataset, test_dataset = split_dataset(images_list)

# Carregamento dos dados
data_loaders, dataset_sizes, class_names, device = resnet18_data_preprocessing([train_dataset, test_dataset], root_path, batch_size=8, workers=0, device='cuda:0')

print("Dataset sizes:", dataset_sizes)
print("Class names:", class_names)
print(data_loaders)

Dataset sizes: {'train': 160, 'test': 40}
Class names: [0, 1]
{'train': <torch.utils.data.dataloader.DataLoader object at 0x00000292998AC190>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x00000292AC62BB20>}


In [26]:
def train_model(model, dataloaders, dataset_sizes, device, save_path, criterion, optimizer, scheduler, num_epochs=25):
    
    '''
        Torch training loop:
    '''

    # Start timer:
    since = time.time()
    print('#################################################################################')
    print(f'Start training model:')
    print('#################################################################################')

    # Initial values:
    best_acc       = 0.0
    best_loss      = 20.0
    best_model_wts = copy.deepcopy(model.state_dict())

    # Loop in number of epochs:
    for epoch in range(num_epochs):
        print('#################################################################################')
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('#################################################################################')

        # Each epoch has a training and validation phase
        for phase in ['train']:
            
            print(f'Current Phase: {phase}.')
            
            model.train()  # Set model to training mode                

            running_loss     = 0.0
            running_corrects = 0

            # Iterate over data in dataloaders:
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward -> track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs  = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss     = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Update loss statistics
                running_loss     += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            scheduler.step()
                
            ################################################################################################
            # Calculate epoch loss:
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc  = running_corrects.double() / dataset_sizes[phase]
            
            # Iteration Output:
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            #################################################################################################
           
    # Training Iteration loop Completed: 
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights and return:
    model.load_state_dict(best_model_wts)
    
    return model

In [27]:
from torchvision import models
from torch.optim import lr_scheduler

import torch.nn as nn
import torch.optim as optim

learning_rate  = 0.00045
momentum       = 0.9
lr_decay_gamma = 0.1
lr_decay_step  = 7
save_path      = root_path + "/resnet18"
num_epochs     = 10


# ResNet18  Fit Config:
model_ft    = models.resnet18(pretrained=True)
num_ftrs    = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))
model_ft    = model_ft.to(device)
criterion   = nn.CrossEntropyLoss()

# Parameters optimization:
optimizer_ft = optim.SGD(model_ft.parameters(), lr=learning_rate, momentum=momentum)

# Learning Rate Decay by a factor of gamma every step_size epochs:
lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=lr_decay_step, gamma=lr_decay_gamma)


In [28]:
# Training, model fit:
model_ft = train_model(model_ft,
                       data_loaders,
                       dataset_sizes,
                       device, 
                       save_path, 
                       criterion, 
                       optimizer_ft, 
                       lr_scheduler, 
                       num_epochs)

#################################################################################
Start training model:
#################################################################################
#################################################################################
Epoch 0/9
#################################################################################
Current Phase: train.
train Loss: 0.6088 Acc: 0.7000
#################################################################################
Epoch 1/9
#################################################################################
Current Phase: train.
train Loss: 0.3541 Acc: 0.8438
#################################################################################
Epoch 2/9
#################################################################################
Current Phase: train.
train Loss: 0.1425 Acc: 0.9813
#################################################################################
Epoch 3/9
#########################################

In [32]:
def infer_model(model, data_loader, device):
    model.eval()  # Coloca o modelo em modo de avaliação
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Desabilita o cálculo do gradiente para economia de memória
        for inputs, labels in data_loader['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calcula a precisão
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy, all_preds, all_labels

In [34]:
from sklearn.metrics import accuracy_score

def infer_model(model, data_loader, device):
    model.eval()  # Coloca o modelo em modo de avaliação
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Desabilita o cálculo do gradiente para economia de memória
        for inputs, labels in data_loader['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calcula a precisão
    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy, all_preds, all_labels

model_ft = model_ft.to(device)  # Move o modelo para o dispositivo

# Realiza a inferência
test_accuracy, test_preds, test_labels = infer_model(model_ft, data_loaders, device)

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Opcional: mostra algumas predições
for i in range(len(test_preds)):
    print(f'Image {i+1} - Predicted: {test_preds[i]}, Actual: {test_labels[i]}')


Test Accuracy: 50.00%
Image 1 - Predicted: 0, Actual: 0
Image 2 - Predicted: 0, Actual: 0
Image 3 - Predicted: 0, Actual: 1
Image 4 - Predicted: 0, Actual: 1
Image 5 - Predicted: 0, Actual: 0
Image 6 - Predicted: 0, Actual: 1
Image 7 - Predicted: 0, Actual: 0
Image 8 - Predicted: 0, Actual: 1
Image 9 - Predicted: 0, Actual: 1
Image 10 - Predicted: 0, Actual: 0
Image 11 - Predicted: 0, Actual: 0
Image 12 - Predicted: 0, Actual: 1
Image 13 - Predicted: 0, Actual: 1
Image 14 - Predicted: 0, Actual: 0
Image 15 - Predicted: 0, Actual: 1
Image 16 - Predicted: 0, Actual: 0
Image 17 - Predicted: 0, Actual: 1
Image 18 - Predicted: 0, Actual: 0
Image 19 - Predicted: 0, Actual: 1
Image 20 - Predicted: 0, Actual: 1
Image 21 - Predicted: 0, Actual: 1
Image 22 - Predicted: 0, Actual: 1
Image 23 - Predicted: 0, Actual: 1
Image 24 - Predicted: 0, Actual: 0
Image 25 - Predicted: 0, Actual: 1
Image 26 - Predicted: 0, Actual: 0
Image 27 - Predicted: 0, Actual: 0
Image 28 - Predicted: 0, Actual: 0
Image 2